## Предсказываеие исходов футбольных матчей

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

import sklearn.base

Вот этот датасет использовлся для обучения модели: https://www.kaggle.com/datasets/martj42/international-football-results-from-1872-to-2017

In [5]:
base = '/Users/sosiska_killer/Documents/ML/archive/'

In [47]:
results = pd.read_csv(base + 'results.csv')
shootouts = pd.read_csv(base + 'shootouts.csv')
goalscorers = pd.read_csv(base + 'goalscorers.csv')

In [21]:
results.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,False
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,False
3,1875-03-06,England,Scotland,2,2,Friendly,London,England,False
4,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,False


Для начала немножечко проанализируем датасет и поищем неточности от которых мы должны избавиться.

In [10]:
results.isnull().sum()
results.dtypes

date          object
home_team     object
away_team     object
home_score     int64
away_score     int64
tournament    object
city          object
country       object
neutral         bool
dtype: object

Поскольку название команд и турниров и даты это категориальные данные то стоит их закодировать. Для навзаний команд буду использовать **Label Encoding**. А с кодированием названий турниров все не так просто. Очеь логично будет предположить, что исход матча очень зависит от значимости турнира. Поэтому я решил остаовиться на **Target Encoding**. Но тут возникает еще один моемнт Target Encoding требует что бы у меня уже был в датасета стобец с результатом встерчи мной было принято решенеие, что победа(домашней команды) - 1, поражение - 0, ничья - 0,5. Для преобразования времени я использовал создание отдельных признаков для года месяца и дня, для того что бы модель более подробно воспринимала времени года к победам. Столбцы с городом и страной я удаляю так как они нам при обучении не пригодятся.


In [48]:
from sklearn.preprocessing import LabelEncoder


# Кодирование названий команд
le = LabelEncoder()
results['home_team'] = le.fit_transform(results['home_team'])
results['away_team'] = le.fit_transform(results['away_team'])

# Кодирование названий турниров
results['outcome'] = results.apply(
    lambda x: 1 if x['home_score'] > x['away_score'] else (0 if x['home_score'] < x['away_score'] else 2),
    axis=1
)
# Вычисление среднего значения целевой переменной для каждого турнира
tournament_mean = results.groupby('tournament')['outcome'].mean()
# Применение кодирования к турнирам
results['tournament_encoded'] = results['tournament'].map(tournament_mean)
# # Удаление столбца с турнирами
results.drop('tournament', axis=1, inplace=True)

In [49]:
# Преобразование строки в datetime
results['date'] = pd.to_datetime(results['date'])

# Создание отдельных признаков для года, месяца и дня
results['year'] = results['date'].dt.year
results['month'] = results['date'].dt.month
results['day'] = results['date'].dt.day

# Удалите исходный столбец даты
results.drop('date', axis=1, inplace=True)

In [50]:
# Удаление названий городов и стран
results.drop('city', axis=1, inplace=True)
results.drop('country', axis=1, inplace=True)

In [51]:
results.head(10)

,home_team,away_team,home_score,away_score,neutral,outcome,tournament_encoded,year,month,day
0,250,89,0,0,False,2,0.97368,1872,11,30
1,88,244,4,2,False,1,0.97368,1873,3,8
2,250,89,2,1,False,1,0.97368,1874,3,7
3,88,244,2,2,False,2,0.97368,1875,3,6
4,250,89,3,0,False,1,0.97368,1876,3,4
5,250,305,4,0,False,1,0.97368,1876,3,25
6,88,244,1,3,False,0,0.97368,1877,3,3
7,310,244,0,2,False,0,0.97368,1877,3,5
8,250,89,7,2,False,1,0.97368,1878,3,2
9,250,305,9,0,False,1,0.97368,1878,3,23


Разделим данные на обучающую и тестовые выборки 

In [52]:
X = results.drop(['outcome', 'home_score', 'away_score'], axis=1) 
y = results['outcome']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

Для начала я хотел бы использовать случайный лес и посмотреть какие он выдаст результаты и если модель будет предсказывтаь больше 80% событий то можно считать что все сделанно успешно 

In [53]:
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)


RandomForestClassifier(random_state=42)

In [54]:
y_pred = rf_model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


Accuracy: 0.49506302521008405
Classification Report:
               precision    recall  f1-score   support

           0       0.43      0.32      0.37      2688
           1       0.54      0.78      0.64      4666
           2       0.26      0.11      0.15      2166

    accuracy                           0.50      9520
   macro avg       0.41      0.40      0.39      9520
weighted avg       0.45      0.50      0.45      9520

Confusion Matrix:
 [[ 853 1570  265]
 [ 659 3628  379]
 [ 450 1484  232]]


In [ ]:
Модель получилось мягко говоря не очень пожалуй нужно либо поменять ее саму либо добавить еще параметров.